In [2]:
from scipy.stats import binom
from texttable import Texttable
import math

Notations

* `f` - presumed number of malicious nodes
* `c` - expected number of nodes qualified for backers
* `N` - total number of nodes

`F` computes a probability of a committee having at least `d` backers completely controlled by malicious nodes

In [38]:
def F(d, c, f, N):
    x = 0
    p = c / N
    for i in range(d, f):
        x += binom.pmf(i, f, p)
    return x

ds = [1,2,3,4,5,6]
f, c, N = 21, 8, 101
t = Texttable()
t.set_cols_dtype(['i','a'])
t.add_row(['d', 'F'])
for d in ds:
    t.add_row([d, F(d, c, f, N)])

print(t.draw())

+---+-------+
| d | F     |
+---+-------+
| 1 | 0.823 |
+---+-------+
| 2 | 0.504 |
+---+-------+
| 3 | 0.229 |
+---+-------+
| 4 | 0.080 |
+---+-------+
| 5 | 0.022 |
+---+-------+
| 6 | 0.005 |
+---+-------+


Compute the probability of adversaries being able to predict the beacon of the next `k`th epoch if using the smallest `m` betas from a block with at least `d` backers to generate the beacon

In [8]:
def B0(m, d, c, N):
    sum = 0
    for i in range(d, N):
        sum += binom.pmf(i, N, c/N) * math.comb(i-d+m, m)
    return sum

def B(a, m, d, c, N):
    ret = a / (1 - a)
    ret *= B0(m, d, c, N)
    return ret

def P(a, m, d, c, N):
    y1 = 0
    y2 = 0
  
    for i in range(d, N):
        x1 = 0
        for j in range(m, i-d+m):
            x1 += binom.pmf(j, i, a) * math.comb(j, m) * math.comb(i-d+m, j) / math.comb(i, j)
        y1 += binom.pmf(i, N, c/N) * x1

    for i in range(d, N):
        x2 = 0
        for j in range(m, i):
            x2 += binom.pmf(j, i, a) * math.comb(i-m, j-m) / math.comb(i, j)
        y2 += binom.pmf(i, N, c/N) * x2

    return y1*a + y2*(1-a)

def M(a, m, d, c, N):
    p = P(a, m, d, c, N)
    return p / (1 - p) 

def Pd(k, a, m, d, c, N):
    return B(a, m, d, c, N) * M(a, m, d, c, N)**(k-1)

t = Texttable()
t.set_cols_dtype(['i', 'i', 'e', 'e', 'e', 'e', 'e'])
t.add_row(['d', 'm', 'B', 'M', 'Pd_2', 'Pd_3', 'Pd_4'])

a, c, N = 20/101, 8, 101
ms = [3]
ds = [5]

for d in ds:
    for m in ms:     
        t.add_row([d, m, B(a,m,d,c,N), M(a,m,d,c,N), Pd(2,a,m,d,c,N), Pd(3,a,m,d,c,N), Pd(4,a,m,d,c,N)])

print(t.draw())

+---+---+-----------+-----------+-----------+-----------+-----------+
| d | m | B         | M         | Pd_2      | Pd_3      | Pd_4      |
+---+---+-----------+-----------+-----------+-----------+-----------+
| 5 | 3 | 9.741e+00 | 4.644e-02 | 4.524e-01 | 2.101e-02 | 9.755e-04 |
+---+---+-----------+-----------+-----------+-----------+-----------+
